**最优化理论**

很多机器学习算法最后都归结为求解最优化问题。最优化问题一般都是求解极大值或者极小值，对于连续可微函数，在数学上有明确的[求解方法](https://blog.csdn.net/lanchunhui/article/details/51371444)。**实际问题中，函数本身可能过于复杂，比如包含指数函数、对数函数以及根号等，使得求解偏导数等于0的方程组不是那么容易，所以我们只能通过迭代的方法求近似解，称为数值计算。**

迭代法从一个初始点 $X_{0}$ 开始，通过某个映射 h 从 $X_{k}$ 移动到下一个点 $X_{k+1}$ ，构造数列 $\{X_k\}_{k\in (1,n)}$，直到收敛到梯度为0的点处。即有下面的极限成立：

$$ \lim_{k \rightarrow + \infty} \nabla f(x_k) = 0$$


一般会利用一阶导数信息即梯度或者二阶导数信息即Hessian矩阵。迭代法的核心是得到迭代公式：

$$x_{k+1} = h(x_k)$$


# 梯度下降法

## 梯度下降法原理

**梯度下降法是通过沿着梯度下降（n=1对应的就是导数）的方向来求解近似极小值。**

我们下面针对一元**连续可微凸函数（也是应用梯度下降法的前提，分段连续可微是否可行呢？理论上也是可行的，分段求解即可）**（即必然有最小值）来推导梯度下降法的一般公式（一种解释[链接](http://www.cnblogs.com/ljygoodgoodstudydaydayup/p/7274943.html)，多元情形请参考[链接](https://zhuanlan.zhihu.com/p/36902908)）。一元函数的一阶泰勒公式展开为：

$$f(x + \Delta x) = f(x) + f^\prime(x)\Delta x + o(\Delta x)$$

即 
$$f(x + \Delta x) - f(x) = f^\prime(x)\Delta x + o(\Delta x)$$

为了保证每次迭代使得 $f(x + \Delta x) - f(x) < 0$ ,要有 $ f^\prime(x)\Delta x < 0 $ 。因为凸函数的梯度 $f^\prime(x) > 0 $，所以要保证 $\Delta x < 0$。

那么每次 $\Delta f = f(x + \Delta x) - f(x)$ 下降多少取决于 $|f^\prime(x)\Delta x|$ 的大小。对于一个固定的点 $ f^\prime(x) $ 是常数，所以 $\Delta f$ 的大小由 $\Delta x $ 决定。不妨设

$$ \Delta x = - \alpha f^\prime(x),\quad \alpha \in (0, 1)$$

则函数下降有：

$$ f(x + \Delta x) - f(x) = - \alpha f^\prime(x)^2$$

下降速度取决于每点的梯度大小以及学习率。

$ \Delta x = - \alpha f^\prime(x)$ 假设的合理性在于：
1. 保证了 $\Delta x $ 和 $ f^\prime(x) $方向相反，_**沿着梯度相反方向下降是最快的方向**_（多元情形下，两者都是向量，夹角是$\cos \theta$）；
2. $\alpha$ 作为学习率控制了下降的幅度，取值在0到1之间，是因为如果步长太大，会越过极小值。

下面是我自己画的草图，注意到 $f^\prime(x) = \tan \theta$ ， 考虑到 

$$ \tan \theta \approx \theta \quad \theta \rightarrow 0$$

当靠近极小值的时候，有$ \Delta x = - \alpha \theta$， 最终会收敛到极小值。

最终，求解极小值点的问题变成了按照图中斜率导数所示的“梯子”方向逐步下降直到最低点的问题。

<img src="figure/梯度下降.png" width="400" hegiht="300" align=left />

实际求解中会遇到局部极小值和鞍点问题，相应的解决办法我们稍后再谈。

## 梯度下降法

对于函数 $f: R^n\rightarrow R$，我们要最小化这个函数。

在 $\mathbf{u}$（单位向量）方向的方向导数是函数在 f 在 $\mathbf{u}$ 方向的斜率，即 $f(\mathbf{x}+\alpha\mathbf{u})$ 关于 $\alpha$ 在 $\alpha=0$ 时的导数，由链式法则可知 $\alpha=0$ 时，

$$\frac{\partial}{\partial\alpha}f(\mathbf{x}+\alpha\mathbf{u}) = \mathbf{u}^T\nabla_{\mathbf{x}}f(\mathbf{x})$$

为了最小化 f，我们希望找到使得 f 下降最快的方向。计算方向导数：

$$\begin{split}
&\min_{\mathbf{u}^T\mathbf{u}=1}\mathbf{u}^T\nabla_{\mathbf{x}}f(\mathbf{x})\\
=& \min_{\mathbf{u}^T\mathbf{u}=1}||\mathbf{u}||_2 ||f(\mathbf{x})||_2\cos{\theta}
\end{split}$$

其中 $\theta$ 是 $\mathbf{u}$ 与梯度的夹角。将 $$||\mathbf{u}||_2=1$$ 带入，忽略与 $\mathbf{u}$ 无关的项，可以简化得到

$$\min_{\mathbf{u}}\cos{\theta}$$

在 $\mathbf{u}$ 与梯度方向相反时取得最小。

换句话说，梯度向量指向上坡，负梯度向量指向下坡。我们在负梯度方向移动可以减小 f。这就是 **梯度下降**。

梯度下降建议新的点为：

$$\mathbf{x}^\prime = \mathbf{x} - \epsilon \nabla_{\mathbf{x}}f(\mathbf{x})$$

其中 $\epsilon$ 为 **学习率**，是一个确定步长大小的正标量。

编程语言实现如下：
1. 确定学习率和初始的参数起点，学习率不能过大，否则会无法收敛；过小会导致计算太多，算法效率低；
2. 迭代：根据梯度下降参数迭代公式，算出新的参数向量，得到新的预测函数，进而得到新的成本；
3. 确认成本函数是否收敛：对比成本函数的变化，看成本是否越来越小，如果两次成本差值小于误差范围，停止迭代，否则继续。

## SGD 和 MBGD

***
### SGD


上面的梯度下降法中每次循环更新参数需要遍历所有的训练数据集，我们可以改成每次随机从训练数据集里抽取一个数据进行迭代计算，这就是**随机梯度下降法(Stochastic Gradient Descent，简称SGD)**，公式如下：

$$\theta_j = \theta_j - \alpha(h(\textbf{x}^{(i)})-y^{(i)})x_j^{(i)}$$

**随机梯度下降算法和梯度下降算法在数学上证明是等价的。**

SGD的优缺点：
* 对于训练数据size比较大的训练集，训练速度快，大概是 O(n) 和 $O(2^n)$ 的对比；
* 伴随的问题是噪音要多，使得SGD并不是每次迭代都向着整体最优化方向。但是由于更新多轮，整体方向还是大致朝着极小值方向更新。
* SGD和MBGD相比BGD,优点是提升了效率，但算法最终不会收敛到最优点，会在最优点附近波动。

为了更加接近最优值，可以逐渐降低学习率：

**开始时，走的每一步较大（这有助于快速前进同时跳过局部最小值），然后变得越来越小，从而使算法到达全局最小值**。 这个过程被称为**模拟退火。**

***
### MBGD

小批量梯度下降法（Mini-batch Gradient Descent，简称MBGD）是 BGD 和 SGD 的折中，为了解决批梯度下降法训练速度慢，以及随机梯度下降法的准确性综合而来,每次迭代选用m个小批量样本进行更新。

下面的算法展示了 沿着这个梯度的估计下降。
***
**MBGD** 小批量随机梯度下降在第 k 个训练迭代的更新
***
**Require:** 学习率 $\epsilon_k$ 和初始参数 $\mathbf{\theta}$

**While** 停止准则未满足 **do**
* 从训练集中采样包含 m 个样本的小批量集合 $\{\mathbf{x}_1,\mathbf{x}_2,\cdots,\mathbf{x}_m \}$,其中 $\mathbf{x}_i$ 对应的值为 $\mathbf{y}_i$;


* 计算梯度估计： $\hat{\mathbf{g}}\leftarrow +\frac{1}{m}\nabla_{\mathbf{\theta}}\sum_i L(f(\mathbf{x}_i;\mathbf{\theta}),\mathbf{y}_i)$;


* 应用更新：$\mathbf{\theta}\leftarrow \mathbf{\theta} - \epsilon \hat{\mathbf{g}}$

**end While**
***

SGD 及其变种算法的关键参数是**学习率**。之前我们介绍 SGD 使用固定的学习率，在实践中有必要随着时间点的推移逐渐降低学习率，因此我们记第 k 步迭代的学习率为 $\epsilon_k$。原因如下：

    当我们使用全批量梯度下降到达极小点时，整个代价函数的真实梯度会变得很小直到为0，所以全批量梯度下降可以使用固定的学习率。但是 MBGD 中梯度估计引入的噪声源(m 个训练样本的随即采样) 并不会在极小值点消失，所以学习率要逐渐降低。

H.Herbert 和 S.Monro 在论文 [A Stochastic Approximation Method](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586) 中给出了保证 SGD及其变种收敛的充分条件是：

$$\begin{split}
& \sum_{k=1}^{\infty}\epsilon_k = \infty\\
& \sum_{k=1}^{\infty}\epsilon_k^2 = \infty
\end{split}$$

实践中，一般会选出**线性衰减学习率** 直到第 $\tau$ 次迭代：

$$\epsilon_k = (1-\alpha)\epsilon_0 + \alpha\epsilon_\tau = \epsilon_0 -\alpha(\epsilon_0 -\epsilon_\tau) $$

其中 $\alpha = \frac{k}{\tau}$。$\tau$ 步迭代后，一般使 $\epsilon$ 保持常数。

备注：线性衰减不满足SGD收敛的充分条件？？

**$\epsilon_0$、$\epsilon_\tau$、$\tau$ 的选择：**

* $\tau$ 通常被设为可以满足反复遍历训练集几百次的迭代次数，如果是SGD，则迭代次数是训练集规模与 $k*100$ 的乘积；


* $\epsilon_\tau$ 一般被设置为 $\epsilon_0*1\%$;


* 如果 $\epsilon_0$ 过大，学习曲线将剧烈震荡，代价函数会明显增加；相比剧烈震荡而言，温和的震荡是良好的，容易在训练随机代价函数时出现，例如使用 Dropout 的代价函数；如果学习率太小，学习过程会很缓慢，并且容易卡在一个相当高的代价值。

    如果初始学习率太低，学习会卡在一个相当高的代价值。就总训练时间和最终代价值而言，最优的初始学习率的效果会好于一般的学习率(??)大约迭代100次之后最佳的效果。所以通常最好是检测最早的几轮迭代，选择一个比在效果上表现最佳的学习率（？？）更大的学习率，但又不能太大导致严重的震荡，更像是一门艺术。

***
**衡量收敛率**


优化算法的收敛速率通过衡量**额外误差 $J(\mathbf{\theta})-\min_{\mathbf{\theta}}J(\mathbf{\theta})$**，即当前代价函数超出最低可能代价的数量。

SGD 应用于凸问题时，k 步迭代之后的额外误差量级是 $O(\frac{1}{\sqrt{k}})$（考虑 m 个均值的标准差为 $\frac{\sigma}{\sqrt{m}}$），强凸情形下是 $O(\frac{1}{k})$。如果想要进一步改进，需要假定额外的前提条件。

全批量梯度下降在理论上比随机梯度有更好的收敛率，但是 Cramer-Rao 界限指出，**泛化误差的下降速度不会快于 $O(\frac{1}{\sqrt{k}})$。** 另外，渐近分析没有考虑 SGD 在下降效率上的优点：**对于大数据集， SGD 只需要非常少量样本计算梯度从而实现初始快速更新，在时间上远远超过了其缓慢的渐近收敛。**

为了权衡全批量梯度下降和 SGD 两者的优点，我们可以再学习过程中逐渐增大 “小批量” 的大小。想要了解更多 SGD，可以参看 [Online Learning and Stochastic
Approximations](https://leon.bottou.org/publications/pdf/online-1998.pdf)

## 动量与 Nesterov 动量

### 动量

虽然 SGD 很受欢迎，但有时学习过程会很慢。**动量方法旨在加速学习，特别是处理高曲率、小但一致的梯度，或者是带噪声的梯度。动量算法积累了之前梯度指数级衰减的移动平均(??)，并且继续沿该方向移动。**如下图所示。

<img src='figure/momentum0.png'>

形式上看，动量算法引入了变量 $\mathbf{v}$ 充当速度角色——代表参数在参数空间移动的方向和速率。速度被设为负梯度的指数衰减平均。超参数 $\alpha\in [0,1)$ 决定了之前梯度的贡献衰减得有多快。更新规则如下： 

$$\begin{split}
& \mathbf{g}\leftarrow \epsilon\nabla_{\mathbf{\theta}}\left(\frac{1}{m}\sum_{i=1}^{m}L(f(\mathbf{x}_i;\mathbf{\theta}), \mathbf{y}_i)\right)\\
& \mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}\\
& \mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}+\mathbf{v}_k
\end{split}$$

其中速度的更新等价于
$$\begin{split}
&\mathbf{v}_k - \frac{\epsilon\mathbf{g}}{\alpha-1} = \alpha(\mathbf{v}_{k-1} - \frac{\epsilon\mathbf{g}}{\alpha-1})\\
& \mathbf{v}_k = -\left(\frac{1-\alpha^k}{1-\alpha}\epsilon\mathbf{g} +\alpha^k\mathbf{v}_0\right)
\end{split}\tag{Mom-1}$$

由(Mom-1)的第二个式子可知，当 k 足够大时，因为$\alpha\in [0,1)$ 所以 

$$\mathbf{v}_k\rightarrow -\frac{1}{1-\alpha}\epsilon\mathbf{g}  \quad when \;\alpha^k\rightarrow 0$$

进而参数的更新为

$$\mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}-\frac{\epsilon\mathbf{g}}{1-\alpha}$$

相比之前的 SGD，速度更新的步长只是学习率乘以梯度范数 $\mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}-\epsilon\mathbf{g}$,现在步长取决于梯度序列的大小和排列($\{\mathbf{g}_k\}$)。

上面的推导是基于连续的梯度指向相同的方向，此时步长最大。也就是说，如果动量算法总是观测到梯度 $\mathbf{g}$,则会在 $-\mathbf{g}$ 上不停加速，直到达到最终速度。其步长大小为

$$\frac{\epsilon||\mathbf{g}||}{1-\alpha}$$

**动量的超参数 $\alpha$ 选择**


因此，将动量的超参数视为 $\frac{1}{1-\alpha}$ 有助于理解。例如 $\alpha=0.9$ 最大对应着梯度下降算法的10倍速度。实践中 $\alpha$ 一般取值为 0.5、 0.9 和 0.99。和学习率一样，$\alpha$ 也会随着时间不断调整。一般初始值是一个较小的值，随后慢慢变大。**随着时间推移调整 $\alpha$ 没有收缩 $\epsilon$ 重要（？？）。**

注：实际迭代时每一步的梯度 $\mathbf{g}_k$ 是不同的，所以步长中系数并不是简单的 $\frac{\epsilon}{1-\alpha}$，这只是一种极端情况，也就是连续的梯度指向相同的方向。

动量算法的算法伪代码图如下：

***
**动量算法** 使用动量的随机梯度下降
***
**Require:** 学习率 $\epsilon$，动量参数 $\alpha$

**Require:** 初始参数 $\mathbf{\theta}$，初始速度 $\mathbf{v}$

**While** 停止准则未满足 **do**
* 从训练集中采样包含 m 个样本的小批量集合 $\{\mathbf{x}_1,\mathbf{x}_2,\cdots,\mathbf{x}_m \}$,其中 $\mathbf{x}_i$ 对应的值为 $\mathbf{y}_i$;


* 计算梯度估计： $\mathbf{g}\leftarrow \frac{1}{m}\nabla_{\mathbf{\theta}}\sum_i L(f(\mathbf{x}_i;\mathbf{\theta}),\mathbf{y}_i)$;


* 计算速度更新： $\mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}$


* 应用更新：$\mathbf{\theta}_k\leftarrow \mathbf{\theta}_{k-1} + \mathbf{v}_k$

**end While**
***

### Nesterov 动量

受到 Nesterov 加速梯度算法的启发， Sutskever 提出了动量算法的一个变种，相应的更新规则如下：

$$\begin{split}
& \mathbf{g}\leftarrow \epsilon\nabla_{\mathbf{\theta}}\left(\frac{1}{m}\sum_{i=1}^{m}L(f(\mathbf{x}_i;\mathbf{\theta}+ a\mathbf{v}), \mathbf{y}_i)\right)\\
& \mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}\\
& \mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}+\mathbf{v}_k
\end{split}$$

可以看出，两者的区别体现在梯度计算上。在 Nesterov 动量中，梯度计算在施加当前速度之后。因此，Nesterov 动量可以解释为往标准动量方法中添加了一个**校正因子**。完整的Nesterov 动量 如下所示。

***
**Nesterov 动量算法** 使用Nesterov 动量的随机梯度下降
***
**Require:** 学习率 $\epsilon$，动量参数 $\alpha$

**Require:** 初始参数 $\mathbf{\theta}$，初始速度 $\mathbf{v}$

**While** 停止准则未满足 **do**

* 从训练集中采样包含 m 个样本的小批量集合 $\{\mathbf{x}_1,\mathbf{x}_2,\cdots,\mathbf{x}_m \}$,其中 $\mathbf{x}_i$ 对应的值为 $\mathbf{y}_i$;


* 应用临时更新： $\mathbf{\hat{\theta}}\leftarrow \mathbf{\theta}+ \alpha\mathbf{v}$;


* 计算梯度估计： $\mathbf{g}\leftarrow \frac{1}{m}\nabla_{\mathbf{\hat{\theta}}}\sum_i L(f(\mathbf{x}_i;\mathbf{\hat{\theta}}),\mathbf{y}_i)$;


* 计算速度更新： $\mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}$


* 应用更新：$\mathbf{\theta}_k\leftarrow \mathbf{\theta}_{k-1} + \mathbf{v}_k$

**end While**
***

在 凸批量梯度的情况下， Nesterov 动量将 额外误差收敛率从 $O(\frac{1}{k})$ （k 步后）改进到 $O(\frac{1}{k^2})$,但是在随机梯度的情况下， Nesterov 动量没有改进收敛率。

# 牛顿法

牛顿法是最广泛使用的二阶方法，它是基于二阶泰勒级数展开在某点$\mathbf{\theta}_0$ 附近来近似 $J(\mathbf{\theta})$ 的优化方法。忽略高阶导数：

$$J(\mathbf{\theta}) \approx J(\mathbf{\theta}_0) + (\mathbf{\theta}-\mathbf{\theta}_0)^T\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)+\frac{1}{2}(\mathbf{\theta}-\mathbf{\theta}_0)^T\mathbf{H}(\mathbf{\theta}-\mathbf{\theta}_0)$$

其中 $\mathbf{H}$ 是 J 相对于 $\mathbf{\theta}$ 的 Hessian 矩阵在 $\mathbf{\theta}_0$ 处的估计。我们求解这个函数的临界点：

$$\frac{\partial J(\mathbf{\theta})}{\partial\mathbf{\theta}} = \mathbf{0}$$

得到

$$\mathbf{\theta}^\ast = \mathbf{\theta}_0-\mathbf{H}^{-1}\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)$$

因此，对于局部的二次函数(具有正定的)$\mathbf{H}$，用 $\mathbf{H}^{-1}$ 重新调整梯度，牛顿法直接跳到极小值。

如果目标函数是凸的但是非二次的(有高阶项)，该更新是迭代的，得到和牛顿法相关的算法。算法如下所示。

<img src='figure/newton0.png'>

在深度学习中，目标函数通常非凸，如鞍点。这种情况我们可以通过正则化 Hessian 矩阵来避免，常用的正则化策略包括在 Hessian 矩阵对角线上增加常数 $\alpha$ 。加入正则化后解变为

$$\mathbf{\theta}^\ast = \mathbf{\theta}_0-[\mathbf{H}+\alpha\mathbf{I}]^{-1}\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)$$

这个正则化策略用于牛顿法的近似，只要 Hessian 矩阵的负特征值比较接近零，效果还是会很好。

曲率方向极端的情况下， $\alpha$ 的值必须足够大，以抵消负特征值。然而，如果 $\alpha$ 持续增加， Hessian 矩阵会变成由 $\alpha\mathbf{I}$ 主导，通过牛顿法选择的方向会收敛到普通梯度的$\frac{1}{\alpha}$ 。 

***
**牛顿法的计算负担**

牛顿法用于训练大型神经网络还受限于明显的计算负担， Hessian 矩阵中元素数目是参数数量的平方。如果参数数目为 k，牛顿法要计算 $k\times k$ 矩阵的逆，计算复杂度为 $O(K^3)$。

另外，由于参数每次更新都会改变，每次训练迭代都要计算 Hessian 矩阵的逆。这就导致，只有参数很小的网络才能在实际中用牛顿法进行训练。

# 共轭梯度法

# BFGS

Broyden–Fletcher–Goldfarb–Shanno（BFGS）算法具有牛顿法的一些优点，但没有牛顿法的计算负担。

我们回顾牛顿法，由下式给出
$$\mathbf{\theta}^\ast = \mathbf{\theta}_0-\mathbf{H}^{-1}\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)$$

其中 $\mathbf{H}$ 是 J 相对于 $\mathbf{\theta}$ 的 Hessian 矩阵在 $\mathbf{\theta}_0$ 处的估计。

运用牛顿法的主要计算难点在于计算 Hessian 逆 $\mathbf{H}^{-1}$。拟牛顿法所采用的方法(BFGS是其中最突出的)是使用矩阵 $\mathbf{M}_t$ 近似逆，迭代更新以更好近似 $\mathbf{H}^{-1}$。

当 Hessian 逆近似 $\mathbf{M}_t$ 更新时，下降方向 $\mathbf{M}_t\mathbf{g}_t$。参数更新为

$$\mathbf{\theta}_{t+1} = \mathbf{\theta}_{t}+\epsilon^\ast\mathbf{M}_t\mathbf{g}_t$$
$\epsilon^\ast$ 是搜索步长。

该方法和共轭梯度不同的是：

* 该方法的成功并不严重依赖于线搜索寻找该方向上和真正极小值很近的一点；


* 相比于 共轭梯度，BFGS 的优点是其花费较少的时间改进每个线搜索；


* BFGS 算法必须存储 Hessian 逆矩阵 $\mathbf{M}$,需要 $O(n^2)$ 的存储空间，使 BFGS 不适用于大多数具有百万级参数的现代深度学习模型。

***
**存储受限的 BFGS(或L-BFGS)** 通过避免存储完整的 Hessian 逆矩阵 $\mathbf{M}$， BFGS 算法的存储代价可以显著降低。L-BFGS 算法使用和 BFGS 算法相同的方法计算 $\mathbf{M}$ 的近似，但是其实假设 $\mathbf{M}^{(t-1)}$ 是单位矩阵，而不是一步一步存储近似。

# [最大似然估计（maximum likelihood estimation，缩写为MLE）](https://zh.wikipedia.org/wiki/%E6%9C%80%E5%A4%A7%E4%BC%BC%E7%84%B6%E4%BC%B0%E8%AE%A1)

## 二分类问题对应的定义

令 $P(Y=1|X=x) = P(x;\theta)$, 其中 P 是以 $\theta$ 为参数的函数。假设样本 $x_i$ 之间相互独立，则似然函数(likelihood function)为

$$L(\theta) = \prod_{i=1}^m P(Y=y_i|X=x_i) = \prod_{i=1}^m P(x_i;\theta)^{y_i}(1-P(x_i;\theta))^{(1- y_i)}$$

对 $L(\theta)$取对数，则得到**对数似然函数**（log-likelihood function）如下：

$$\ell(\theta) = \ln L(\theta) = \sum_{i=1}^{m} y_i \ln P(x_i;\theta) + \sum_{i=1}^{m} (1-y_i) \ln (1-P(x_i;\theta))$$

称

$$ \theta^{\star} = \arg\max_{\theta}\ell(\theta) = -\arg\min_{\theta}\ell(\theta)$$

为**最大化对数似然估计**.

## 最大似然函数与最小化交叉熵的等价性

考虑一组含有 m 个样本的数据集 $\mathbf{X} = \{\mathbf{x}_1,\mathbf{x}_2, \cdots,\mathbf{x}_m\}$，由未知的真实数据分布 $p_{data}(\mathbf{x})$ 生成。

令 $p_{model}(\mathbf{x};\mathbf{\theta})$ 是一族由 $\mathbf{\theta}$ 确定在相同空间上的概率分布。换言之，$p_{model}(\mathbf{x};\mathbf{\theta})$ 是 模型或者神经网络模型对 $p_{data}(\mathbf{x})$ 的逼近。

则 $\mathbf{\theta}$ 的最大似然估计被定义为

$$\mathbf{\theta}_{ML} = \arg\max_{\mathbf{\theta}}p_{model}(\mathbf{X};\mathbf{\theta})=\arg\max_{\mathbf{\theta}}\prod_{i=1}^m p_{model}(\mathbf{x}_i;\mathbf{\theta})$$

多个数值的乘积容易导致数值下溢，我们发现似然对数不会改变 $\arg\max$，我们将乘积通过对数函数转换为求和形式：

$$\mathbf{\theta}_{ML} = \arg\max_{\mathbf{\theta}}\sum_{i=1}^{m}\log p_{model}(\mathbf{x}_i;\mathbf{\theta})$$

因为缩放代价函数不会改变 $\arg\max$，我们可以除以 m 得到和训练数据经验分布 $\hat{p}_{data}(\mathbf{x})$ 相关的期望作为准则：

$$\mathbf{\theta}_{ML} = \arg\max_{\mathbf{\theta}}E_{\mathbf{x}\sim \hat{p}_{data}}\log p_{model}(\mathbf{x};\mathbf{\theta})$$

注：[大数定律](https://zh.wikipedia.org/wiki/%E5%A4%A7%E6%95%B0%E5%AE%9A%E5%BE%8B)可以支持以上转换的成立。

一种解释最大似然估计的观点是，将它看做最小化训练集上的经验分布 $\hat{p}_{data}$ 和模型分布之间的差异(**也就是模型分布尽可能和经验分布 $\hat{p}_{data}$ 相匹配**)，两者之间的差异可以通过 KL 散度度量：

$$D_{KL}(\hat{p}_{data}||p_{model}) =E_{x\sim \hat{p}_{data}}\;[\log \hat{p}_{data}(\mathbf{x})-\log p_{model}(\mathbf{x})]$$

等式右侧第一项不涉及模型，意味着训练模型最小化 KL 散度时，只需要最小化

$$-E_{x\sim \hat{p}_{data}}\;\log p_{model}(\mathbf{x})$$

所以**最大化对数似然等价于最小化 KL 散度，也等价于最小化交叉熵。**

# [Lagrange 对偶与KKT条件](https://wizardforcel.gitbooks.io/dm-algo-top10/content/svm-2.html)

对于 向量 $\mathbf{x}\in R^n$，考虑一般形式的不等式约束问题：

$$\left\{\begin{aligned}
\min_{\mathbf{x}} f(\mathbf{x}) &\\
s.t. h_i(\mathbf{x}) = 0 &\quad i = 1,2,...,k \\
g_i(\mathbf{x}) \leq  0 &\quad i = 1,2,...,l
\end{aligned}\right. \tag{Primal Constraints}$$

对应的 Lagrange 公式为：

$$L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = f(\mathbf{x}) + \sum_{i=1}^{k}\alpha_i g_i(\mathbf{x}) + \sum_{i=1}^{l}\beta_i h_i(\mathbf{x}) \tag{Larange Model}$$

其中 $\mathbf{\alpha}, \mathbf{\beta}$ 都是 Lagrange 算子。

我们要求解的就是满足 **primal constraints (简称PC)** 的 $\min_{\mathbf{x}}f(\mathbf{x})$。

对于 $L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta})$ 我们适当地调整 $\mathbf{\alpha}, \mathbf{\beta}$ 的值，总会有：

$$ \max_{\beta,\alpha \geq 0} L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta})=
\left \{
\begin{aligned}
f(\mathbf{x}) & \quad (\mathbf{\alpha}, \mathbf{\beta}) \; satisfy \; (PC)\\
+\infty & \quad else 
\end{aligned}\right.$$

如果记 $\theta_{PC}(\mathbf{x}) = \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0} L(x, \mathbf{\alpha}, \mathbf{\beta})$，则求解满足 PC 的 $\min_{\mathbf{x}}f(\mathbf{x})$的问题等价于

$$\begin{split}
d^\ast = \min_{\mathbf{x}}f(\mathbf{x}) =& \min_{\mathbf{x}} \theta_{PC}(\mathbf{x})\\
=& \min_{\mathbf{x}} \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0} L(\mathbf{x}, \alpha, \beta)
\end{split}$$

一般有 $\max\min f \leq \min\max f$ (从 $f \leq \max f$开始推导) ,所以有：

$$ d^\ast = \min_{\mathbf{x}}f(\mathbf{x}) =\min_{\mathbf{x}} \max_{\mathbf{\beta}, \mathbf{\alpha} \geq 0} L(\mathbf{x}, \alpha, \beta)
\geq \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0}\min_{\mathbf{x}} L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = p^\ast$$

到此为止，求解不等式左侧满足约束的极小值问题转换成不等式右侧求极大值的问题。求极大值是凸规划里的常见问题，对偶问题的转换都是往利于求解的方向。

**KKT条件：**

求解满足 （PC）的极小值问题，对应的 Lagrange 公式存在极小值的必要条件是满足KKT条件：

$$\left\{\begin{aligned}
\nabla_{\mathbf{x}}  L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = \mathbf{0}\\
h_i(\mathbf{x}) = 0 \\
g_i(\mathbf{x}) \leq  0  \\
\alpha_i \geq 0 \\
\alpha_i g_i = 0
\end{aligned}\right. \tag{KKT}$$
